In [16]:
import numpy as np
import pandas as pd
import os
import ast
from datetime import datetime
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt

import pdb

import requests
import calendar
import time

In [2]:
path = './TweetScraper-master/Data/newSearchTweets'
tweets_df = pd.DataFrame()
toremove = dict.fromkeys((ord(c) for c in u'/xa0/n/t'))

pbar = tqdm(total=(87491))

for file in os.listdir(path):
    if '.DS_Store' not in file:
        pbar.update(1)
        with open(path+'/'+file, 'rb') as f:
            data = pickle.load(f)
            data['datetime'] = datetime.strptime(data['datetime'], '%Y-%m-%d %H:%M:%S')
            data['ID'] = int(data['ID'])
            tweets_df = tweets_df.append(data, ignore_index=True)
            
pbar.close()

In [3]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85535 entries, 0 to 85534
Data columns (total 13 columns):
ID               85535 non-null float64
datetime         85535 non-null datetime64[ns]
is_reply         85535 non-null float64
is_retweet       85535 non-null float64
nbr_favorite     85535 non-null float64
nbr_reply        85535 non-null float64
nbr_retweet      85535 non-null float64
text             85535 non-null object
url              85535 non-null object
user_id          85535 non-null object
usernameTweet    85535 non-null object
has_media        29059 non-null float64
medias           29059 non-null object
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 8.5+ MB


In [3]:
tweets_df.head()

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias
0,9.885302e+17,2018-04-23 17:29:08,0.0,0.0,0.0,0.0,0.0,New post (Could Ripple XRP Really Reach $1...,/ExcelVisibility/status/988530235493748736,815322320,ExcelVisibility,NaN,NaN
1,9.866292e+17,2018-04-18 11:35:08,0.0,0.0,0.0,0.0,0.0,Long/Short Bitcoin swings with up to 100x Leve...,/KerriHermanx/status/986629205856223233,956303409065414656,KerriHermanx,NaN,NaN
2,9.847927e+17,2018-04-13 09:57:41,0.0,0.0,0.0,0.0,0.0,"Did you know that you can buy, sell, store, co...",/coindirectcom/status/984792745666580481,915453520924545025,coindirectcom,1.0,[https://t.co/h1CTJHGwPs]
3,9.863800e+17,2018-04-17 19:05:02,0.0,0.0,0.0,0.0,0.0,1 Ripple = 0.6589 USD. Ripple has changed ...,/RippleMarket/status/986380040517701633,848989770923601922,RippleMarket,NaN,NaN
4,9.866047e+17,2018-04-18 09:57:42,0.0,0.0,0.0,0.0,0.0,How Is #Ripple Different From All Other #Cryp...,/Matthix191/status/986604686856933376,916641996823031808,Matthix191,1.0,[https://t.co/xF5l49lz8P]


In [4]:
for t in tweets_df['text'].head(25):
    print(t)
    print()
    print()

New post (Could  Ripple   XRP  Really Reach $10.00 In 39 Days?) has been published on  -  https:// masscryptocurrency.com/?p=2664   pic.twitter.com/6Dlog2TNQE


Long/Short Bitcoin swings with up to 100x Leverage at BitMEX!

10% Fee Discount using the link below  

→  http:// bitmex.com/register/m9MNxm   

 $ LSK   $ BCH   $ DGB   $ AION   $ EMC   $ RDD   $ SALT   $ OMG   $ ARK   $ EOS   $ MAID   $ WTC   $ QTUM   $ XRP   $ LTC   $ ELF   $ ZEC   $ BAT pic.twitter.com/IFj1YxJk0O


Did you know that you can buy, sell, store, convert and transfer @Ripple  #XRP  on  http:// Coindirect.com       in the UK, EU countries, Australia, Kenya, Nigeria and South Africa. You can use fiat currency or convert other coins and exchange them for #Ripple  .


1  Ripple  = 0.6589 USD.  Ripple  has changed by -0.0026 USD in 30 mins. Live price:  https:// is.gd/hI9OcA    #ripple  #xrp  #cryptocurrency


How Is #Ripple  Different From All Other #Cryptocurrencies ? An Ultimate Guide  https://www. reddit.com/r/R

In [5]:
tweets_df.to_pickle('./Data/Raw/86kRaw.pkl')

In [22]:
tweets_df = pd.read_pickle('./Data/Raw/86kRaw.pkl')

In [23]:
xrpDelta = pd.read_pickle('./Data/Cleaned/xrp_delta.pkl')

In [24]:
def cielDatetime(d):
    delta = 3600-((d.minute*60+d.second)%3600)
    
    td = dt.timedelta(seconds=delta)
    return d.to_pydatetime()+td

In [25]:
tweets_df['rounded_dateTime'] = tweets_df['datetime'].apply(lambda t : cielDatetime(t))
tweets_df['timeId'] = tweets_df['rounded_dateTime'].apply(lambda t : time.mktime(t.timetuple()))

In [26]:
fullData = pd.merge(tweets_df, xrpDelta, how='left', on='timeId')

In [27]:
fullData.head()

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias,rounded_dateTime,timeId,time,pDelta,change
0,9.885302e+17,2018-04-23 17:29:08,0.0,0.0,0.0,0.0,0.0,New post (Could Ripple XRP Really Reach $1...,/ExcelVisibility/status/988530235493748736,815322320,ExcelVisibility,NaN,NaN,2018-04-23 18:00:00,1.524521e+09,2018-04-23 18:00:00,0.002178,neutral
1,9.866292e+17,2018-04-18 11:35:08,0.0,0.0,0.0,0.0,0.0,Long/Short Bitcoin swings with up to 100x Leve...,/KerriHermanx/status/986629205856223233,956303409065414656,KerriHermanx,NaN,NaN,2018-04-18 12:00:00,1.524067e+09,2018-04-18 12:00:00,0.011943,positive
2,9.847927e+17,2018-04-13 09:57:41,0.0,0.0,0.0,0.0,0.0,"Did you know that you can buy, sell, store, co...",/coindirectcom/status/984792745666580481,915453520924545025,coindirectcom,1.0,[https://t.co/h1CTJHGwPs],2018-04-13 10:00:00,1.523628e+09,2018-04-13 10:00:00,-0.004874,neutral
3,9.863800e+17,2018-04-17 19:05:02,0.0,0.0,0.0,0.0,0.0,1 Ripple = 0.6589 USD. Ripple has changed ...,/RippleMarket/status/986380040517701633,848989770923601922,RippleMarket,NaN,NaN,2018-04-17 20:00:00,1.524010e+09,2018-04-17 20:00:00,0.007330,positive
4,9.866047e+17,2018-04-18 09:57:42,0.0,0.0,0.0,0.0,0.0,How Is #Ripple Different From All Other #Cryp...,/Matthix191/status/986604686856933376,916641996823031808,Matthix191,1.0,[https://t.co/xF5l49lz8P],2018-04-18 10:00:00,1.524060e+09,2018-04-18 10:00:00,-0.014478,negative


In [28]:
fullData.to_pickle('./Data/Cleaned/processed_data.pkl')